<img src="https://upload.wikimedia.org/wikipedia/commons/1/16/Closed_circuit_ventilators.gif"/>
<center>Image Source: Tabletop Whale: Explaining ventilators for COVID-19 http://tabletopwhale.com/2020/04/01/explaining-ventilators-for-covid-19.html</center>

# About Competition
Mechanical ventilation is a clinician-intensive procedure that was prominently on display during the early days of the COVID-19 pandemic. Developing new methods for controlling mechanical ventilators is prohibitively expensive, even before reaching clinical trials. High-quality simulators could reduce this barrier.
 Current simulators are trained as an ensemble, where each model simulates a single lung setting. However, lungs and their attributes form a continuous space, so a parametric approach must be explored that would consider the differences in patient lungs.
 The team at Google Brain aims to grow the community around machine learning for mechanical ventilation control. They believe neural networks and deep learning can better generalize across lungs with varying characteristics.
 In this competition, you’ll simulate a ventilator connected to a sedated patient's lung. The best submissions will take lung attributes compliance and resistance into account.Competition file is available [here](https://www.kaggle.com/c/ventilator-pressure-prediction).

# Load Competition Dataset

Competition dataset located in "/kaggle/input"; This path defined by Kaggle to access the competition file. We will list two files from this path as input files.

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path=os.path.join(dirname, filename)
        if 'train' in path:
            __training_path=path
        elif 'test' in path:
            __test_path=path

## Input Dataset

In [3]:
#loaded files
print(f'Training path:{__training_path}\nTest path:{__test_path}')

In [1]:
# Kaggle Environment Prepration
#update kaggle env
import sys
#you may update the environment that allow you to run the whole code
!{sys.executable} -m pip install --upgrade scikit-learn=="0.24.2"

In [2]:
#record this information if you need to run the Kernel internally
import sklearn; sklearn.show_versions()

# Input Dataset

In [ ]:
def __load__data(__training_path, __test_path, concat=False):
	"""load data as input dataset
	params: __training_path: the training path of input dataset
	params: __test_path: the path of test dataset
	params: if it is True, then it will concatinate the training and test dataset as output
	returns: generate final loaded dataset as dataset, input and test
	"""
	# LOAD DATA
	import pandas as pd
	__train_dataset = pd.read_csv(__training_path, delimiter=',')
	__test_dataset = pd.read_csv(__test_path, delimiter=',')
	return __train_dataset, __test_dataset
__train_dataset, __test_dataset = __load__data(__training_path, __test_path, concat=True)
__train_dataset.head()

In [ ]:
# STORE SUBMISSION RELEVANT COLUMNS
__test_dataset_submission_columns = __test_dataset['id']

### Discard Irrelevant Columns
In the given input dataset there are <b>1</b> column that can be removed as follows:* id *.

In [ ]:
# DISCARD IRRELEVANT COLUMNS
__train_dataset.drop(['id'], axis=1, inplace=True)
__test_dataset.drop(['id'], axis=1, inplace=True)

### Target Column
The target column is the value which we need to predict.
Therefore, we need to detach the target columns in prediction.
Note that if we don't drop this fields, it will generate a model with high accuracy on training and worst accuracy on test (because the value in test dataset is Null).
Here is the list of *target column*: <b>pressure</b>

In [ ]:
# DETACH TARGET
__feature_train = __train_dataset.drop(['pressure'], axis=1)
__target_train =__train_dataset['pressure']
__feature_test = __test_dataset

# Training Model and Prediction
First, we will train a model based on preprocessed values of training data set.
Second, let's predict test values based on the trained model.

## LightGBM Regressor
We will use *LightGBM Regressor* which is constructing a gradient boosting model. We will use *lightgbm* package.
More detail about *LightGBM Regressor* can be found [here](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html).

In [ ]:
# MODEL
import numpy as np
from lightgbm import LGBMRegressor
__model = LGBMRegressor()
__model.fit(__feature_train, __target_train) 
__y_pred = __model.predict(__feature_test)

# Submission File
We have to maintain the target columns in "submission.csv" which will be submitted as our prediction results.

In [ ]:
# SUBMISSION
submission = pd.DataFrame(columns=['id'], data=__test_dataset_submission_columns)
submission['pressure'] = __y_pred
submission.head()

In [ ]:
# save submission file
submission.to_csv("kaggle_submission.csv", index=False)